In [14]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.crs import CRS
from shapely.geometry import Polygon
from tqdm import tqdm

In [15]:

# Example usage:
raster_url = 'assets/sat_img/01_13_23_imagery_clipped.tif'
shapefile_url = 'Temp/shapefile/Ward_29.shp'
output_path = 'Temp/output/clipped_raster.tif'

# Read the raster file
with rasterio.open(raster_url) as src:
    # Get CRS of the raster
    raster_crs = src.crs

    # Read the shapefile
    shapefile_gdf = gpd.read_file(shapefile_url)

    # Check CRS of the shapefile
    if shapefile_gdf.crs != raster_crs:
        # Reproject the shapefile to match the CRS of the raster
        shapefile_gdf = shapefile_gdf.to_crs(raster_crs)

    # Convert the shapefile geometry to list of shapely Polygon objects
    geometries = [Polygon(feature['geometry']) for _, feature in shapefile_gdf.iterrows()]

    # Initialize a list to store clipped raster data
    clipped_rasters = []

    # Iterate over geometries and mask the raster using tqdm to show progress
    for geometry in tqdm(geometries, desc="Clipping Raster", unit="polygon"):
        # Mask the raster using the reprojected shapefile
        raster_data, raster_transform = mask(src, [geometry], crop=True)

        # Append the clipped raster data to the list
        clipped_rasters.append(raster_data)

    # Combine all clipped raster data into one array
    clipped_data = sum(clipped_rasters)

    # Get metadata of clipped raster
    raster_meta = src.meta.copy()

# Update metadata for clipped raster
raster_meta.update({
    'height': clipped_data.shape[1],
    'width': clipped_data.shape[2],
    'transform': raster_transform,
    'crs': raster_crs  # Ensure CRS is preserved
})

# Write the clipped raster to disk
with rasterio.open(output_path, 'w', **raster_meta) as dst:
    dst.write(clipped_data)

Clipping Raster: 100%|██████████| 1/1 [00:03<00:00,  3.66s/polygon]


In [2]:
import os

In [7]:
print(os.environ["GIS_REPORTING_SYSTEM"])

KeyError: 'GIS_REPORTING_SYSTEM'

In [11]:
from docx import Document
from fpdf import FPDF

# Input DOCX file path
docx_file_path = '/Users/Nayan/Desktop/work/GIS_Reporting_System/Temp/output/docx/document.docx'

# Output PDF file path
pdf_file_path = '/Users/Nayan/Desktop/work/GIS_Reporting_System/Temp/output/pdfs/document.pdf'

# Open the DOCX file
doc = Document(docx_file_path)

# Create a PDF object
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Add each paragraph from the DOCX to the PDF
for paragraph in doc.paragraphs:
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, paragraph.text)

# Save the PDF file
pdf.output(pdf_file_path)

print(f"Conversion completed: {pdf_file_path}")


Conversion completed: /Users/Nayan/Desktop/work/GIS_Reporting_System/Temp/output/pdfs/document.pdf
